In [1]:
from threading import Thread
import json
from stmpy import Machine, Driver
import paho.mqtt.client as mqtt
from utils import BROKER, PORT, TOPIC, JOIN_TOPIC, QUEUE_TOPIC, HELP_TOPIC, UPDATE_TOPIC 


class MQTT_Client_1:
    def __init__(self, stm):
        self.state_machine = stm
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.stm_driver: Driver = None
        self.session_id = ""

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        try:
            
            message: dict  = json.loads(msg.payload.decode('utf-8'))
        except json.decoder.JSONDecodeError:
            return
    
   
        
        #print("on_message(): topic: {}".format(msg.topic))
        
        if msg.topic == (f"{TOPIC}/{JOIN_TOPIC}"):
            if message['msg'] == "session_joined":
                print("Correct code, session joined")
                self.stm_driver.send("correct_code", "student")

                #Unsubscribe from the JOIN TOPIC so other messages are'nt received

                self.client.unsubscribe(f"{TOPIC}/{JOIN_TOPIC}")
                print("Sucsessfully unsubscribed to: "+ (f"{TOPIC}/{JOIN_TOPIC}"))
                self.session_id = message['session_id']
                

                # Send the session_id to the student object
                self.state_machine.session_id = message["session_id"]
                
                #Subscribe to the session and the following topics:
                #team02/session_id/HELP_TOPIC
                #team02/session_id/UPDATE_TOPIC
                self.client.subscribe(f"{TOPIC}/{message['session_id']}/{HELP_TOPIC}")
                print("Sucsessfully subscribed to: "+ (f"{TOPIC}/{message['session_id']}/{HELP_TOPIC}"))
                
                self.client.subscribe(f"{TOPIC}/{message['session_id']}/{UPDATE_TOPIC}")
                print("Sucsessfully subscribed to: "+ (f"{TOPIC}/{message['session_id']}/{UPDATE_TOPIC}"))
                self.client.subscribe(f"{TOPIC}/{self.session_id}/{QUEUE_TOPIC}")
            else:
                
                if message['msg'] == "session_join_failed":
                    print("Incorrect code, try again")
                    self.stm_driver.send("wrong_code", "student")
                
                                     

        """ if msg.topic == (f"{TOPIC}/{JOIN_TOPIC}"):
            if message['msg'] == "session_join_failed":
                print("Incorrect code, try again")
                self.stm_driver.send("wrong_code", "student")"""
                
                
        if msg.topic == f"{TOPIC}/{self.session_id}/{QUEUE_TOPIC}":
            print("Receiving queue, passing on to state machine")
            self.state_machine.queue_message = message['queue']
            self.stm_driver.send("queue_request", "student")
            print("queue sent sucsessfully to state machine")
                    
    def start(self, broker, port):
        #Kobler til main topic og join topic og broker
        print("Connectin to {}:{}".format(broker, port))
        self.client.connect(broker, port)
        self.client.subscribe(TOPIC)
        self.client.subscribe(f"{TOPIC}/{JOIN_TOPIC}")

        try:
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()
            
            


In [2]:
!pip3 install paho-mqtt
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import json
from stmpy import Machine, Driver
import ipywidgets as widgets
from IPython.display import display
import random
import sys

#NOTES:
## Håndtere i server som mottar join-koder at den enten sender trigger: riktig kode, eller trigger: feil kode, 
# utifra det så vet vi hvilken state vi skal tils
#Hver eneste student må ha en slags kode/gruppenavn


class Student:
      def __init__(self):
            self.button_create = widgets.Button(description="Join Session")
            self.button_create.on_click(self.on_button_join)
            self.queue_message = []
            self.session_id = ""
            self.mqtt_client: mqtt.Client = None
            self.stm: Machine = None
            self.question = 1
            # MIDLERTIDIG
            self.total_questions = 5
            
            # text field
            self.stu_code = widgets.Text(value='', placeholder='', description='student code:', disabled=False)
            self.code = self.stu_code.value
            display(self.button_create, self.stu_code)
            
            self.group_name_widget = widgets.Text(value='', placeholder='', description='Group name:', disabled=False)
            
            self.group_name = self.group_name_widget.value

            display(self.button_create, self.group_name_widget)
            display()

      def on_wait(self):
            """TODO"""


      
      def ask_help(self):

     
            dictMessage = { "msg": "request_help", "group_name": self.group_name_widget.value, "question": self.question}
            jsonMessage = json.dumps(dictMessage)
            print("Asking for help")
            self.mqtt_client.publish(f"{TOPIC}/{self.session_id}/{HELP_TOPIC}", jsonMessage)
            self.number_queue.value = "Queuing"

      def finish_task(self):
            print(f"Finished with task {self.question}")
            dictMessage = {"msg": "task_finished", "question": self.question, "group_name": self.group_name_widget.value}
            jsonMessage = json.dumps(dictMessage)
            self.mqtt_client.publish(f"{TOPIC}/{self.session_id}/{UPDATE_TOPIC}", jsonMessage)
            if self.question == self.total_questions:
                   self.button_finish_task.disable = True
            else: 
                  self.question += 1
                  self.progress_bar.value += 1 
                  self.button_task.value = self.question
            
            
                  
                     

      def update_queue(self):
            self.place = 1
            if self.group_name_widget.value not in self.queue_message:
                  self.number_queue.value = "Not in queue"
            #ENABLE HELP
            else:
                  for i in self.queue_message:
                        if i == self.group_name_widget.value:
                              break
                        else:
                              self.place += 1  
                  Sondre_idiot_confirmed = str(self.place)             
                  self.number_queue.value = Sondre_idiot_confirmed
            
      
      def publish(self):
            
            dictMessage = {"msg": "join_session","student_code": self.stu_code.value, "group_name": self.group_name_widget.value}
            jsonMessage = json.dumps(dictMessage)

            self.mqtt_client.publish(f"{TOPIC}/{JOIN_TOPIC}" ,jsonMessage)
            self.stu_code.value = "Loading"

      def setup_lab(self):
            print("Setting up lab...")
            self.button_task = widgets.IntText(value= self.question, placeholder='', description='Task', disabled=True)
            display(self.button_task)
            self.button_help = widgets.Button(description="Get help")
            self.button_help.on_click(self.on_button_ask_help)
            display(self.button_help)
            self.button_finish_task = widgets.Button(description="Finish task")
            self.button_finish_task.on_click(self.on_button_finish_task)
            display(self.button_finish_task)
            self.progress_bar = widgets.IntProgress(min = 1, max = self.total_questions, description = "Progress")
            display(self.progress_bar)
            self.button_leave = widgets.Button(description="Leave Session")
            self.button_leave.on_click(self.on_button_leave)
            display(self.button_leave)
            self.number_queue = widgets.Text(value="Not in queue", placeholder='', description='Queue nr.', disabled=True)
            display(self.number_queue)
            print("Lab set up")
            
            pass

      def wrong_code(self):
            self.button_create = widgets.Button(description="Join Session")
            self.button_create.on_click(self.on_button_join)
            self.queue_message = []
            self.session_id = ""
            self.question = 1
            # MIDLERTIDIG
            self.total_questions = 5
            
            # text field
            self.stu_code = widgets.Text(value='', placeholder='', description='student code:', disabled=False)
            self.code = self.stu_code.value
            display(self.button_create, self.stu_code)
            
            self.group_name_widget = widgets.Text(value='', placeholder='', description='Group name:', disabled=False)
            
            self.group_name = self.group_name_widget.value

            display(self.button_create, self.group_name_widget)
            self.mqtt_client.subscribe(f"{TOPIC}/{JOIN_TOPIC}")
            display()


      def save_text(self):
           self.code = self.stu_code.value
           

      def on_button_join(self, session_code_stu):
        self.stm.send('join') # <---- here we send a message
        pass
  
      def on_button_finish_task(self, session_code_stu):
            self.stm.send('finish_task')
            pass
      
      def on_button_ask_help(self, session_code_stu):
            self.stm.send('ask_help')
            pass
      def on_button_leave(self, session_code_stu):
            self.stm.send('leave_lab')
            leave_message = {"msg": "leave_session", "group_name": self.group_name_widget.value}
            json_message = json.dumps(leave_message)
            self.mqtt_client.publish(f"{TOPIC}/{self.session_id}/{UPDATE_TOPIC}",json_message)
            pass
      
# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions

#t1 and t2 are transitions from idle to wait and back
t1 = {'trigger':'join', 
      'source':'idle', 
      'target':'wait'}


t2 = {'trigger':'wrong_code', 
      'source':'wait', 
      'target':'idle',
      'effect': 'wrong_code'}


t3 = {'trigger':'correct_code', 
      'source':'wait', 
      'target':'lab_session_active',
      'effect': 'setup_lab'}


t4 = {'trigger':'ask_help', 
      'source':'lab_session_active', 
      'target':'lab_session_active',
      'effect': 'ask_help' }

t5 = {'trigger':'leave_queue', 
      'source':'lab_session_active', 
      'target':'lab_session_active',
      'effect': 'leave_queue'}

t6 = {'trigger':'finish_task', 
      'source':'lab_session_active', 
      'target':'lab_session_active',
      'effect': 'finish_task'}

#Trengs ikke siden det skjer automatisk når man ber om hjelp
t7 = {'trigger':'queue_request', 
       'source':'lab_session_active', 
       'target':'lab_session_active',
       'effect': 'update_queue'
       }


t8 = {'trigger':'leave_lab', 
      'source':'lab_session_active', 
      'target':'idle',
      'effect': 'wrong_code'
      }




# the states:
idle = {'name': 'idle'}

wait = {'name': 'wait',
        'entry': 'publish'}

lab_session_active = {'name': 'lab_session_active'}

    

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [ ]:
student = Student()
state_machine = Machine(transitions=[t0, t1, t2, t3, t4, t5, t6, t7, t8], states=[idle, wait, lab_session_active], obj=student, name="student")
student.stm = state_machine

driver = Driver()
driver.add_machine(state_machine)

myclient = MQTT_Client_1(student)
student.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(BROKER, PORT)

Button(description='Join Session', style=ButtonStyle())

Text(value='', description='student code:', placeholder='')

Button(description='Join Session', style=ButtonStyle())

Text(value='', description='Group name:', placeholder='')

Connectin to mqtt20.iik.ntnu.no:1883


...

Button(description='Finish task', style=ButtonStyle())

IntProgress(value=1, description='Progress', max=5, min=1)

Button(description='Leave Session', style=ButtonStyle())

Text(value='Not in queue', description='Queue nr.', disabled=True, placeholder='')

Lab set up
Receiving queue, passing on to state machine
queue sent sucsessfully to state machine
Receiving queue, passing on to state machine
queue sent sucsessfully to state machine
on_connect(): Connection Accepted.
